# GPT from Scratch

Trying to jump in a bit deeper on language models, this code is just me studying from a video that Karpathy made:
https://youtu.be/kCc8FmEb1nY?si=4S4voHUOZv2Dyrn1

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

### Getting and cleaning the training data

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-22 23:34:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M   295KB/s    in 3.7s    

2024-08-22 23:34:19 (295 KB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(len(text.split()))

202651


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Encoder and Decoder at Character Level

In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda string: [stoi[ch] for ch in string]
decode = lambda string: ''.join([itos[ch] for ch in string])

print(encode('hello world!'))
print(decode(encode('hello world!')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
hello world!


In [10]:
# enoding the entire text dataset into torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [11]:
#Splitting into training and validation
n = int(0.9* len(data))
train = data[:n]
validation = data[n:]

### Training the transformer

In [12]:
# There are 8 individual patterns packed here, which explains the +1
block_size = 8
train[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train[:block_size]
y = train[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} then the target:{target}")

when input is tensor([18]) then the target:47
when input is tensor([18, 47]) then the target:56
when input is tensor([18, 47, 56]) then the target:57
when input is tensor([18, 47, 56, 57]) then the target:58
when input is tensor([18, 47, 56, 57, 58]) then the target:1
when input is tensor([18, 47, 56, 57, 58,  1]) then the target:15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then the target:47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then the target:58


In [16]:
# Batch Size (Another Dimension)

torch.manual_seed(1337)
batch_size = 4 # Number of independent sequences can we process in parallel
block_size = 8 # Maximum context length for our predictions

def get_batch(split):
    data = train if split == 'train' else validation
    ix = torch.randint(len(data) - block_size, (batch_size,)) # randomly generated 4 numbers b/w 0 and block size
    x = torch.stack([data[i:i+block_size] for i in ix]) # get chunks for all of the ix
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # get the target for the tensors
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

# 32 total examples in a single batch of size 4
for b in range(batch_size): # Batch Dimension
    for t in range(block_size): # Time Dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} then the target:{target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] then the target:43
when input is [24, 43] then the target:58
when input is [24, 43, 58] then the target:5
when input is [24, 43, 58, 5] then the target:57
when input is [24, 43, 58, 5, 57] then the target:1
when input is [24, 43, 58, 5, 57, 1] then the target:46
when input is [24, 43, 58, 5, 57, 1, 46] then the target:43
when input is [24, 43, 58, 5, 57, 1, 46, 43] then the target:39
when input is [44] then the target:53
when input is [44, 53] then the target:56
when input is [44, 53, 56] then the target:1
when input is [44, 53, 56, 1] then the target:58
when input is [44, 53,

In [17]:
nn.Embedding(vocab_size, vocab_size)

Embedding(65, 65)

In [29]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        
        super().__init__()
        # each token reads off the logits for the next token from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensors of integers
        
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape # Reshaping

            logits = logits.view(B*T,C)

            targets = targets.view(B*T)

            loss = F.cross_entropy(logits,targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # predictions
            logits, loss = self(idx)
            # focus only on the last step -> bigram
            logits = logits[:,-1,:] # Makes it (B,C)
            # apply softmax
            probs = F.softmax(logits, dim=-1) # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx
            
            
    
m = BigramLanguageModel(vocab_size)
# we are predicting what comes next based on individual identity of a single token
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [30]:
idx = torch.zeros((1,1), dtype=torch.long)
idx

tensor([[0]])

# totally random modeling

In [35]:
decode(m.generate(idx, max_new_tokens = 100)[0].tolist())

'\nrVchSFUIdd q?sPJpUdhMCK$VXXevXJFMl,i\nYxA:gWId,EXR,iMC,$?srV$VztRwb?KpgUWFjR$zChOLm;JrDnDph\nLBj,KZxJa'

In [47]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1.2e-3)

In [48]:
batch_size = 64

for steps in range(10000):
    xb,yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    
    optimizer.zero_grad(set_to_none=True)
    
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.439894199371338


In [50]:
print(decode(m.generate(idx, max_new_tokens = 100)[0].tolist()))


YO, t milll ll thepa'djo henose Fomevofan GLOfoouloredoolu ket.
Wine d urus, lomere by,
ADod at alse


### printing french lmao

Karapthy's math trick for self attention

In [54]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [61]:
x[0][7] # 2 channels and 8 time components (8 tokens in a batch)

tensor([-0.9211,  1.5433])

The token should only flow information backwards, do an average of all the tokens before, which summarizies the information behind, but this is lossy but works ish for now.

In [140]:
bag = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        prev = x[b,:t+1]
        bag[b,t] = torch.mean(prev,0)

In [141]:
bag[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [142]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

we can achieve this efficiently using mathematication

In [143]:
a = torch.tril(torch.ones(3,3))
a

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [144]:
a = a / torch.sum(a,1, keepdim=True)
a

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

Just doing regular average of all words before the target word in the batch

In [148]:
wei = torch.tril(torch.ones(T,T))
wei = wei / torch.sum(wei,1, keepdim=True)
bag2 = wei @ x

In [149]:
torch.allclose(bag2,bag)

True

### Self attention mechanism

Now we do not want all the tokens to be the same importance, some might be more important, so we will add key, query in the tranformer. Below we will implement the single head for self attention.

In [185]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# head size is a hyper parameter
head_size = 16
key = nn.Linear(C,head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)


k = key(x) # (B,T,16)
q = query(x)  # (B,T,16)

wei = q @ k.transpose(-2,-1) # transpose the last two dimensions (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) * (head_size ** -0.5)
wei = F.softmax(wei, dim=-1)

v = value(x)

out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [186]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
         [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
         [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
         [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4016, 0.5984, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3365, 0.2271, 0.4364, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3019, 0.2060, 0.2899, 0.2022, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1058, 0.1700, 0.1530, 0.3451, 0.2261, 0.0000, 0.0000, 0.0000],
         [0.1526, 0.164

In [200]:
class Head(nn.Module):
    def __init__(self, block_size,head_size):
        super().__init__()
        self.key = nn.Linear(C,head_size, bias=False)
        self.query = nn.Linear(C, head_size, bias=False)
        self.value = nn.Linear(C, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
    
    def forward(self,x):
        B,T,C = x.shape
        k = key(x) # (B,T,C)
        q = query(x)  # (B,T,C)
        wei = q @ k.transpose(-2,-1) # transpose the last two dimensions (B, T, 16) @ (B, 16, T) --> (B, T, T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) * (head_size ** -0.5)
        wei = F.softmax(wei, dim=-1)
        v = value(x)
        out = wei @ v
        return out

        

In [202]:
torch.manual_seed(1337)
n_embed = 32
block_size = 8

class BigramLanguageModel(nn.Module):
    
    def __init__(self,vocab_size):
        
        super().__init__()
        # each token reads off the logits for the next token from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
        self.sa_head = Head(block_size=block_size,head_size=n_embed)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensors of integers
        token_embeddings = self.token_embedding_table(idx) # (B,T,C)
        position_embeddings = self.position_embedding_table(torch.arange(idx.shape[1]))
        x = token_embeddings + position_embeddings
        x = self.sa_head(x) # apply self attention head here
        logits = self.lm_head(x)
        
        if targets == None:
            loss = None
        else:
            B,T,C = logits.shape # Reshaping

            logits = logits.view(B*T,C)

            targets = targets.view(B*T)

            loss = F.cross_entropy(logits,targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            idx_cond = idx[:, -block_size:]
            
            # predictions
            logits, loss = self(idx_cond)
            # focus only on the last step -> bigram
            logits = logits[:,-1,:] # Makes it (B,C)
            # apply softmax
            probs = F.softmax(logits, dim=-1) # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx

# Batch Size (Another Dimension)

torch.manual_seed(1337)

def get_batch(split):
    data = train if split == 'train' else validation
    ix = torch.randint(len(data) - block_size, (batch_size,)) # randomly generated 4 numbers b/w 0 and block size
    x = torch.stack([data[i:i+block_size] for i in ix]) # get chunks for all of the ix
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # get the target for the tensors
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')
    
m = BigramLanguageModel(vocab_size)
# we are predicting what comes next based on individual identity of a single token
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

inputs:
torch.Size([64, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
      

RuntimeError: mat1 and mat2 shapes cannot be multiplied (512x16 and 32x65)